In [1]:
import time
import os
import pprint
import logging
from pyspark import SparkContext, SQLContext

from code.common import *

In [2]:
!pkill -9 java

In [2]:
from pyspark import SparkContext, SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
logging.getLogger("py4j").setLevel(logging.ERROR)

## Oversample Data

In [6]:
train_file = 'data/train.parquet.normed.masked-060000.encode.picked-000987.packed'
dev_file = 'data/valid.parquet.normed.masked-060000.encode.picked-000987.packed'

In [7]:
train = sqlContext.read.parquet(train_file)

#### Count the distribution across the labels

In [12]:
label_counts_df = train.groupby('label').count().toPandas()
label_counts_df

,label,count
0,1,9394612
1,0,27274974


In [13]:
negative_count = label_counts_df.at[0, 'count']
positive_count = label_counts_df.at[1, 'count']

In [15]:
total_count = positive_count + negative_count
print(f'Total number of train examples is {total_count}')
print (f'Percentage of negative examples is {negative_count*100/total_count}')
print (f'Percentage of postive examples is {positive_count*100/total_count}')

Total number of train examples is 36669586
Percentage of negative examples is 25.619629302605162
Percentage of postive examples is 74.38037069739484


In [16]:
extra_positives = train.filter(train['label']==1).sample(withReplacement=True, fraction=1.9)

In [17]:
extra_positives.show(5)

+-----+--------------------+------------------+
|label|            features|            weight|
+-----+--------------------+------------------+
|    1|(1000,[0,1,2,4,5,...|0.7438037069739484|
|    1|(1000,[0,1,2,4,5,...|0.7438037069739484|
|    1|(1000,[0,1,2,4,5,...|0.7438037069739484|
|    1|(1000,[0,1,2,4,5,...|0.7438037069739484|
|    1|(1000,[0,1,2,4,5,...|0.7438037069739484|
+-----+--------------------+------------------+
only showing top 5 rows



##### Add the extra positive examples and shuffle

In [18]:
oversampled_train = train.union(extra_positives)
oversampled_train = oversampled_train.sample(withReplacement = False, fraction = 1.0)

In [19]:
oversampled_train.write.parquet('data/train.parquet.normed.masked-060000.encode.picked-000987.packed.oversampled')

#### Tree Based Models

In [4]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [5]:
train_file = 'data/train.parquet.normed.masked-060000.encode.picked-000987.packed-001000.oversampled'
dev_file = 'data/valid.parquet.normed.masked-060000.encode.picked-000987.packed'

In [6]:
train = sqlContext.read.parquet(train_file)

In [7]:
estimator = DecisionTreeClassifier(labelCol="label", featuresCol = "features")
    
print('Starting training')
start_train = time()
model = estimator.fit(train)
end_train = time()
print('Finished training')

Starting training


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o28.fit

#### Decison Tree Classifier

In [5]:
def train_estimator(estimator, train_file, dev_file, labelCol="label", featuresCol="features", *args, **kwargs):
    
    train = sqlContext.read.parquet(train_file)
    
    algorithm_name = estimator.__name__
    
    estimator = estimator(labelCol=labelCol, featuresCol = featuresCol, *args, **kwargs)
    
    print('Starting training')
    start_train = time()
    model = estimator.fit(train)
    end_train = time()
    print('Finished training')
    
    dev = sqlContext.read.parquet(dev_file)
    
    print('Making predictions')
    train_predictions = model.transform(train)
    test_predictions = model.transform(dev)
    end_predictions = time()
    print('Compeleted making predictions')

    # Calculate the AUC for train and dev
    auc_train = evaluator.evaluate(train_predictions)
    auc_dev = evaluator.evaluate(test_predictions)

    print(f'{algorithm_name} - AUC on train is: {auc_train * 100:.2f}')
    print(f'{algorithm_name} - AUC on dev is: {auc_dev * 100:.2f}')
    print(f'Time to train is {(end_train-start_train)/60:.2f} minutes')
    print(f'Time to predict is {(end_predictions-end_train)/60:.2f} minutes')

In [6]:
train_estimator(estimator=DecisionTreeClassifier, train_file=train_file, dev_file=dev_file)

Starting training


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43337)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py

Py4JError: An error occurred while calling o28.fit

#### Random Forests

In [7]:
train_estimator(estimator=RandomForestClassifier, train_file=train_file, dev_file=dev_file)

Starting training


Py4JJavaError: An error occurred while calling o28.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 232, localhost, executor driver): java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:65)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:332)
	at java.nio.channels.Channels.writeFullyImpl(Channels.java:78)
	at java.nio.channels.Channels.writeFully(Channels.java:101)
	at java.nio.channels.Channels.access$000(Channels.java:61)
	at java.nio.channels.Channels$1.write(Channels.java:174)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.Buffer

In [ ]:
train.